In [3]:
pip install imblearn

     |████████████████████████████████| 199 kB 20.3 MB/s            
     |████████████████████████████████| 189 kB 36.9 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [6]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import roc_curve, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from imblearn.over_sampling import RandomOverSampler
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler


# Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445-wagner'
bucket = s3.Bucket(bucket_name)

# Defining the file to be read from s3 bucket
file_key = "Iris.csv"

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading the csv file
iris = pd.read_csv(file_content_stream)
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [10]:
#frequency table of species
iris['Species'].value_counts()

Iris-versicolor    50
Iris-virginica     50
Iris-setosa        50
Name: Species, dtype: int64

In [11]:
#creating species_numb
iris['Species_numb']=np.where(iris['Species'] == 'Iris-virginica',1, np.where(iris['Species'] == 'Iris-versicolor', 2, 3))
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,Species_numb
0,1,5.1,3.5,1.4,0.2,Iris-setosa,3
1,2,4.9,3.0,1.4,0.2,Iris-setosa,3
2,3,4.7,3.2,1.3,0.2,Iris-setosa,3
3,4,4.6,3.1,1.5,0.2,Iris-setosa,3
4,5,5.0,3.6,1.4,0.2,Iris-setosa,3


In [12]:
#Defining variables
X = iris[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
Y = iris['Species_numb']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.2, stratify = Y)

In [13]:
#Generating data
X_train, X_test, Y_train, Y_test = train_test_split(X , Y, test_size = 0.2, stratify = Y)

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [14]:
#Creating random forest model
RF = OneVsRestClassifier(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3)).fit(X_train, Y_train)
RF_pred = RF.predict_proba(X_test)
RF_pred = np.argmax(RF_pred, axis = 1) + 1
confusion_matrix(Y_test, RF_pred)

array([[10,  0,  0],
       [ 3,  7,  0],
       [ 0,  0, 10]])

In [15]:
#Creating support vector machine model
SVM = OneVsRestClassifier(estimator = SVC(kernel = 'rbf', probability = True)).fit(X_train, Y_train)
SVM_pred = SVM.predict_proba(X_test)
SVM_pred = np.argmax(SVM_pred, axis = 1) + 1
confusion_matrix(Y_test, SVM_pred)

array([[10,  0,  0],
       [ 0, 10,  0],
       [ 0,  0, 10]])

In [ ]:
## Based on the two above models the SVM model is the one I would use